Importanção das bibliotecas

In [1]:
import dataprocessing.datacolect as dc
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from urllib.error import HTTPError, URLError
import pandas as pd
import numpy as np

# 1. Sobre a coleta no ***Metacritic***

# 2. Rotina de ***scraping***

In [2]:
url = ("https://www.metacritic.com/browse/game/xbox-one/all/all-time/new/"
      "?platform=xbox-one&releaseYearMin=1910&releaseYearMax=2023&page=")

É necessário coletar o *link* para cada jogo em cada uma das páginas do ***Metacritic*** referentes ao ***Xbox One***. Há, para essa plataforma, 319 páginas.

In [4]:
for page in range(1, 3):
    # soup = dc.Soup(url + str(page))
    # soup = soup.get_soup()
    # print(soup)
    print(url + str(page))

https://www.metacritic.com/browse/game/xbox-one/all/all-time/new/?platform=xbox-one&releaseYearMin=1910&releaseYearMax=2023&page=1
https://www.metacritic.com/browse/game/xbox-one/all/all-time/new/?platform=xbox-one&releaseYearMin=1910&releaseYearMax=2023&page=2


In [ ]:
# for page in range(1, 2):
#     soup = dc.Soup(url + str(page))
#     soup = soup.get_soup()

#     scrape = dc.Scrape()
#     game_cards = scrape.find_elements(
#         tag='div', tag_class='c-finderProductCard c-finderProductCard-game',
#         timeout=0.2, soup=soup
#     )

#     for game in game_cards:
#         game_links = scrape.find_element(
#             tag='a', tag_class='c-finderProductCard_container g-color-gray80 u-grid',
#             timeout=0.2, soup=game_cards
#         )
#         print(game_links)
